In [1]:
import torch
import torch.nn as nn

# from models.common import *
from yolov7_utils.common import Conv, Concat, MP, SPPCSPC, RepConv, SP # add SP
from nas.supernet.yolo import IDetect
from yolov7_utils.general import make_divisible
from copy import deepcopy

# libraries from ofa
from nas.supernet.dynamic_layers import DynamicConvLayer, DynamicConv2d, DynamicBatchNorm2d
from nas.supernet.search_block import ELAN, ELANBlock, BBoneELAN, HeadELAN, DyConv, TinyELAN, ELAN2 # add tinyelan

# ELAN block test

-> ELANBlock 상위 클래스 만들어서 기본 메소드 상속하는 방식

* BBoneELAN

In [2]:
input = torch.randn(1, 128, 64, 64)
block = BBoneELAN(c1=128, c2=64, k=3, depth=4)

block(input).shape, block(input, d=3).shape, block(input, d=2).shape, block(input, d=1).shape

(torch.Size([1, 320, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 192, 64, 64]),
 torch.Size([1, 128, 64, 64]))

* Dynamic Convolution 검증

In [3]:
# c2 * (depth + 1)
dyconv = DyConv(64*(4+1), 256, 3, 1)
dyconv(block(input)).shape, dyconv(block(input, d=3)).shape, dyconv(block(input, d=2)).shape, dyconv(block(input, d=1)).shape

(torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]))

In [4]:
# TODO we can even search the out channel size
dyconv.conv.active_out_channel = 32
dyconv(input).shape

torch.Size([1, 32, 64, 64])

* Head ELAN Block

In [5]:
input = torch.randn(1, 512, 64, 64)
block = HeadELAN(c1=512, c2=256, k=3, depth=5)

block(input).shape, block(input, d=4).shape, block(input, d=3).shape, block(input, d=2).shape, block(input, d=1).shape

(torch.Size([1, 1024, 64, 64]),
 torch.Size([1, 896, 64, 64]),
 torch.Size([1, 768, 64, 64]),
 torch.Size([1, 640, 64, 64]),
 torch.Size([1, 512, 64, 64]))

* Dynamic Convolution 검증

In [6]:
# c2 // 2 * (depth + 3)
dyconv = DyConv(256//2*(5+3), 256, 3, 1)

dyconv(block(input)).shape, dyconv(block(input, d=3)).shape, dyconv(block(input, d=2)).shape, dyconv(block(input, d=1)).shape

(torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]))

In [7]:
# we can even search the out channel size
dyconv.conv.active_out_channel = 32
dyconv(input).shape

torch.Size([1, 32, 64, 64])

* Tiny ELAN Block

In [112]:
input = torch.randn(1, 128, 64, 64)
block = TinyELAN(c1=128, c2=64, k=3, depth=4, act=nn.LeakyReLU())
block(input).shape, block(input, d=4).shape, block(input, d=3).shape, block(input, d=2).shape, block(input, d=1).shape

(torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 192, 64, 64]),
 torch.Size([1, 128, 64, 64]))

In [9]:
block = TinyELAN(c1=128, c2=64, k=3, depth=4, act=nn.LeakyReLU()) # nn.SiLU(), nn.ReLU()
block

TinyELAN(
  (act): LeakyReLU(negative_slope=0.01)
  (layers): Sequential(
    (0): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): LeakyReLU(negative_slope=0.01)
    )
    (1): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): LeakyReLU(negative_slope=0.01)
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): LeakyReLU(negative_slope=0.01)
    )
    (3): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act

* Dynamic Convolution 검증

In [10]:
# c2 // 2 * (depth + 3)
dyconv = DyConv(256//2*(5+3), 256, 3, 1)

dyconv(block(input)).shape, dyconv(block(input, d=3)).shape, dyconv(block(input, d=2)).shape, dyconv(block(input, d=1)).shape

(torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]),
 torch.Size([1, 256, 64, 64]))

In [11]:
# we can even search the out channel size
dyconv.conv.active_out_channel = 32
dyconv(input).shape

torch.Size([1, 32, 64, 64])

* get_active_subnet method 검증

In [12]:
# This part is for get_active_subnet in supernet_yolov7.py
b_block = BBoneELAN(c1=128, c2=64, k=3, depth=4)
depth = 3
act_idx = b_block.act_idx[depth]
layers = deepcopy(b_block.layers[:act_idx+1])
block = ELANBlock(b_block.mode, layers, depth)

input = torch.randn(1, 128, 64, 64)
block(input).shape

torch.Size([1, 256, 64, 64])

# YOLOSuperNet

* parse_supernet 함수 정의
: BBoneELAN과 HeadELAN에 대한 c2 계산 방식이 상이함.

In [126]:
def parse_supernet(d, ch):  # model_dict, input_channels(3)
    print('\n%3s%18s%3s%10s  %-40s%-30s' % ('', 'from', 'n', 'params', 'module', 'arguments'))
    anchors, nc, gd, gw = d['anchors'], d['nc'], d['depth_multiple'], d['width_multiple']
    na = (len(anchors[0]) // 2) if isinstance(anchors, list) else anchors  # number of anchors
    no = na * (nc + 5)  # number of outputs = anchors * (classes + 5)

    layers, save, c2 = [], [], ch[-1]  # layers, savelist, ch out
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):  # from, number, module, args
        m = eval(m) if isinstance(m, str) else m  # eval strings
        for j, a in enumerate(args):
            try:
                args[j] = eval(a) if isinstance(a, str) else a  # eval strings
            except:
                pass

        n = max(round(n * gd), 1) if n > 1 else n  # depth gain
        if m in [nn.Conv2d, Conv, RepConv, SPPCSPC, DyConv]:
            c1, c2 = ch[f], args[0]
            if c2 != no:  # if not output
                c2 = make_divisible(c2 * gw, 8)

            args = [c1, c2, *args[1:]]
            if m in [SPPCSPC]:
                args.insert(2, n)  # number of repeats
                n = 1
        elif m is BBoneELAN:
            c1, c2 = ch[f], int(args[0]*(args[-1]+1))
            args = [c1, *args]
        elif m is HeadELAN:
            c1, c2 = ch[f], int((args[0]*2) + (args[0]/2 * (args[-1]-1)))
            args = [c1, *args]
        elif m is TinyELAN: # TinyELAN
            #(c1=128, c2=64, k=3, depth=4, act=nn.LeakyReLU())
            c1, c2 = ch[f], args[0]
            args = [c1, c2, *args[1:]]
        elif m is nn.BatchNorm2d:
            args = [ch[f]]
        elif m is Concat:
            c2 = sum([ch[x] for x in f])
        # elif m is Chuncat:
        #     c2 = sum([ch[x] for x in f])
        # elif m is Shortcut:
        #     c2 = ch[f[0]]
        # elif m is Foldcut:
        #     c2 = ch[f] // 2
        elif m in [IDetect]:
            args.append([ch[x] for x in f])
            if isinstance(args[1], int):  # number of anchors
                args[1] = [list(range(args[1] * 2))] * len(f)
        # elif m is ReOrg:
        #     c2 = ch[f] * 4
        # elif m is Contract:
        #     c2 = ch[f] * args[0] ** 2
        # elif m is Expand:
        #     c2 = ch[f] // args[0] ** 2
        else:
            c2 = ch[f]
        print(m)
        m_ = nn.Sequential(*[m(*args) for _ in range(n)]) if n > 1 else m(*args)  # module
        t = str(m)[8:-2].replace('__main__.', '')  # module type
        np = sum([x.numel() for x in m_.parameters()])  # number params
        m_.i, m_.f, m_.type, m_.np = i, f, t, np  # attach index, 'from' index, type, number params
        print('%3s%18s%3s%10.0f  %-40s%-30s' % (i, f, n, np, t, args))  # print
        save.extend(x % i for x in ([f] if isinstance(f, int) else f) if x != -1)  # append to savelist
        layers.append(m_)
        if i == 0:
            ch = []
        ch.append(c2)
    return nn.Sequential(*layers), sorted(save)

In [148]:
import yaml
#yaml_file = r'C:\Users\Lab13\Documents\GitHub\yolov7-nas-for-TANGO\yaml\yolov7-tiny.yaml'
yaml_file = '/yaml/yolov7-tinyelan.yaml'
with open(yaml_file) as f:
    yaml = yaml.load(f, Loader=yaml.SafeLoader)

# Define model
ch, nc, anchors = 3, None, None

ch = yaml['ch'] = yaml.get('ch', ch)  # input channels
if nc and nc != yaml['nc']:
    print(f"Overriding model.yaml nc={yaml['nc']} with nc={nc}")
    yaml['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    yaml['anchors'] = round(anchors)  # override yaml value

In [149]:
model, save = parse_supernet(deepcopy(yaml), ch=[ch])


                 from  n    params  module                                  arguments                     
<class 'yolov7_utils.common.Conv'>
  0                -1  1       928  yolov7_utils.common.Conv                [3, 32, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  1                -1  1     18560  yolov7_utils.common.Conv                [32, 64, 3, 2, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'nas.supernet.search_block.TinyELAN'>
  2                -1  1     22784  nas.supernet.search_block.TinyELAN      [64, 32, 3, 2, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.Conv'>
  3                -1  1      2176  yolov7_utils.common.Conv                [32, 64, 1, 1, None, 1, LeakyReLU(negative_slope=0.1)]
<class 'yolov7_utils.common.MP'>
  4                -1  1         0  yolov7_utils.common.MP                  []                            
<class 'nas.supernet.search_block.TinyELAN'>
  5                -1  1     82432  nas.sup

In [150]:
from yolov7_utils.common import * 
from nas.supernet.yolo import * 

def forward_once(model, x, runtime_depth, profile=False):
    y, dt = [], []  # outputs
    elan_idx = 0
    for m in model:
        if m.f != -1:  # if not from previous layer
            x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

        if profile:
            c = isinstance(m, (Detect, IDetect, IAuxDetect, IBin))
            o = thop.profile(m, inputs=(x.copy() if c else x,), verbose=False)[0] / 1E9 * 2 if thop else 0  # FLOPS
            for _ in range(10):
                m(x.copy() if c else x)
            t = time_synchronized()
            for _ in range(10):
                m(x.copy() if c else x)
            dt.append((time_synchronized() - t) * 100)
            print('%10.1f%10.0f%10.1fms %-40s' % (o, m.np, dt[-1], m.type))

        if isinstance(m, ELAN): # 
            depth = runtime_depth[elan_idx]
            elan_idx += 1
            x = m(x, d=depth)
        else:
            x = m(x)  # run
        
        y.append(x if m.i in save else None)  # save output

    if profile:
        print('%.1fms total' % sum(dt))
    return x

In [151]:
input = torch.randn(1, 3, 64, 64)
runtime_depth = [3,2,2,1,5,5,5,3]

forward_once(model, input, runtime_depth)

RuntimeError: Given groups=1, weight of size [64, 32, 1, 1], expected input[1, 64, 16, 16] to have 32 channels, but got 64 channels instead

## YOLOSuperNet 검증

In [36]:
import yaml
yaml_file = 'yaml/yolov7_dynamicsupernet.yml'
with open(yaml_file) as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

In [37]:
# Define model
ch, nc, anchors = 3, None, None

ch = config['ch'] = config.get('ch', ch)  # input channels
if nc and nc != config['nc']:
    print(f"Overriding model.yaml nc={config['nc']} with nc={nc}")
    config['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    config['anchors'] = round(anchors)  # override yaml value

In [38]:
model, save = parse_supernet(deepcopy(config), ch=[ch])


                 from  n    params  module                                  arguments                     
<class 'yolov7_utils.common.Conv'>
  0                -1  1       928  yolov7_utils.common.Conv                [3, 32, 3, 1]                 
<class 'yolov7_utils.common.Conv'>
  1                -1  1     18560  yolov7_utils.common.Conv                [32, 64, 3, 2]                
<class 'yolov7_utils.common.Conv'>
  2                -1  1     36992  yolov7_utils.common.Conv                [64, 64, 3, 1]                
<class 'yolov7_utils.common.Conv'>
  3                -1  1     73984  yolov7_utils.common.Conv                [64, 128, 3, 2]               
<class 'nas.supernet.search_block.BBoneELAN'>
  4                -1  1    164608  nas.supernet.search_block.BBoneELAN     [128, 64, 3, 3]               
<class 'nas.supernet.search_block.DyConv'>
  5                -1  1     66048  nas.supernet.search_block.DyConv        [256, 256, 1, 1]              
<class 'yolov7_utils.

In [39]:
def get_active_net_config(yaml, runtime_depth): # self
    idx = 0
    d = deepcopy(yaml)
    
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):
        if 'ELAN'in m:
            args[-1] = runtime_depth[idx]
            idx += 1
    
    del d['depth_list']
            
    return d
    

In [40]:
runtime_depth = [3,2,2,1,5,5,5,3]
d = get_active_net_config(config, runtime_depth)

In [41]:
for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):
    if 'ELAN'in m:
        print(i, m, args)

4 BBoneELAN [64, 3, 3]
11 BBoneELAN [128, 3, 2]
18 BBoneELAN [256, 3, 2]
25 BBoneELAN [256, 3, 1]
32 HeadELAN [256, 3, 5]
38 HeadELAN [128, 3, 5]
45 HeadELAN [256, 3, 5]
52 HeadELAN [512, 3, 3]


In [43]:
from yolov7_utils.common import * 
from nas.supernet.yolo import * 

def forward_once(model, x, runtime_depth, profile=False):
    y, dt = [], []  # outputs
    elan_idx = 0
    for m in model:
        if m.f != -1:  # if not from previous layer
            x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

        if profile:
            c = isinstance(m, (Detect, IDetect, IAuxDetect, IBin))
            o = thop.profile(m, inputs=(x.copy() if c else x,), verbose=False)[0] / 1E9 * 2 if thop else 0  # FLOPS
            for _ in range(10):
                m(x.copy() if c else x)
            t = time_synchronized()
            for _ in range(10):
                m(x.copy() if c else x)
            dt.append((time_synchronized() - t) * 100)
            print('%10.1f%10.0f%10.1fms %-40s' % (o, m.np, dt[-1], m.type))

        if isinstance(m, ELAN): # 
            depth = runtime_depth[elan_idx]
            elan_idx += 1
            x = m(x, d=depth)
        else:
            x = m(x)  # run
        
        y.append(x if m.i in save else None)  # save output

    if profile:
        print('%.1fms total' % sum(dt))
    return x

In [44]:
input = torch.randn(1, 3, 256, 256)
runtime_depth = [3,2,2,1,5,5,5,3]

forward_once(model, input, runtime_depth)

[tensor([[[[[-5.12626e-01, -2.12486e-01,  7.29461e-02,  ..., -9.46013e-02, -1.49151e-01,  3.10560e-01],
            [-5.26625e-01,  2.78568e-01,  1.46677e-01,  ...,  1.89838e-01, -3.96693e-01,  3.62671e-01],
            [ 5.63842e-02, -5.15891e-03, -1.01364e-01,  ...,  1.38700e-01, -2.22295e-01,  1.21738e-01],
            ...,
            [-4.93032e-01, -4.79689e-01, -2.84990e-01,  ..., -5.52354e-03, -9.27650e-02,  2.57976e-01],
            [-1.37420e-01,  5.98946e-02,  3.79577e-01,  ..., -1.55369e-01, -1.88062e-01,  3.04876e-01],
            [-4.11585e-01,  1.79435e-01,  1.39512e-01,  ...,  1.87518e-01, -8.37151e-03,  2.55889e-01]],
 
           [[ 3.80759e-02,  6.04044e-02,  1.64540e-01,  ..., -6.47056e-02,  7.55637e-02,  9.42520e-02],
            [-4.12101e-01, -7.54018e-02,  1.69501e-01,  ...,  8.14262e-02, -1.10328e-01, -2.06994e-02],
            [-1.28312e-01, -1.21234e-01,  1.88257e-01,  ...,  4.35028e-01, -6.61205e-01,  1.90765e-01],
            ...,
            [-5.40518e-01, 

In [45]:
x = torch.randn(1, 3, 256, 256)

y, dt = [], []  # outputs
for i, m in enumerate(model):
    if m.f != -1:  # if not from previous layer
        x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers
        
    x = m(x)  # run
    
    y.append(x if m.i in save else None)  # save output

In [46]:
len(x)

3